In [1]:

import ipdb
import os
import pyomo.environ as pyo
import pandas as pd
import numpy as np
# import pyomo.environ as pyomo
# from pyomo.environ import SolverFactory
# from pyomo.environ import *
#import cplex
from docplex.mp.model import Model
#Model2 = cplex.Cplex()   
#Model2.parameters.mip.tolerances.mipgap.set(float(0.1))
df = pd.read_csv("C:/hybrid_bidding/software_module-20231023T230959Z-001/software_module/data/EPRI_final_analysis/Case 24, 28_ High VRE, High Hyb, Uncstr GC/LBNL_SCH_TMP1011.csv")
#print(df.x__Schedule)
#dates=datetime(2019,7,1,"TimeZone","America/New_York"):datetime(2019,7,31,"TimeZone","America/New_York");
## Input and parameters
cvar_weight = 0 # %0 to 1 with 0=do not consider risk and 1=fully consider risk
gridcharge = 1 #% 0=No grid charging allowed, 1=Grid charging unrestricted
ngenscen_sample=100 #number of scenarios to sample from historical distribution
nscen=200
ngenscen=20 #final number of generation scenarios to consider in bidding
fcast_band=.05 #for generation scenarios, proportion of GEN.max
err_band=.05; #for generation scenarios, proportion of GEN.max
gen_fcast_prob=0.8; #probability of given DA forecast
nprevdays=[14,21]; #weekday, weekend
avg_wmae_tgt=0.05;
#compute bids - may need to update once
bid_type=2; #(2=stairstep, 1=self-schedule)
nscen_breaks=6;
break_mode="mid";
price_floor=-150;
price_ceil=1000;
soc0_ub=[0, 0.05, 0.2, 0.4, 0.6, 0.8, 0.95, 1]; #initial SOC bands
soc0_pos_in_range=0.5; #in (0,1] => (lower bound, upper bound]
x_step_min=0.1; #Treats two points on the bid curve as equal if x (MW) component differs by less than this threshold.
                #Points are removed from curve accordingly.
q1min=1e-10; #smallest positive quantity bid allowed by PSO
#compute bids - should not need to edit
cvar_confid = 0.95;
Time_p=48
eta_plus = 0.5*np.ones(48); #concentrates bidding in DA (don't change)
eta_minus= 1.5*np.ones(48); #concentrates bidding in DA (don't change)
dev_plus_addifzero = -.5; #ensures costs for deviation when price scen is 0
dev_minus_addifzero = .5; #ensures costs for deviation when price scen is 0
p_thresh_heur_pcentile=[0.2,0.8]; #price percentiles (of annual prices) used to set low/high price heuristic
pHoriz_scale=2; #for high/low price heuristic
#Process input data
#[POI,GEN,ESR] = process_PSOdata_config(inj_path,esr_path,hyb_name,gen_name,esr_name);
#[gen_data,gen_col] = process_PSOfile_gen(gendata_path,sched_inj_path,gen_name); 
#[price_data,price_col] = process_PSOfile_2Rprices(pricedata_path,int_time_path,hyb_name, hyb_name);%esr_name,gen_name);
class ESR:
    pass
esr=ESR()
esr.init=30
esr.max=240
esr.min=0
esr.effCrg=0.922
esr.effDis=0.922
esr.opCost=0;
esr.power=60;
#print (esr.max)
gen=ESR()
gen.max =125
gen.opCost=0
POI=185
from process_gen import process_gendata
process_gendata()
PRICE_STEP = pd.read_excel(r"C:\hybrid_bidding\software_module-20231023T230959Z-001\software_module\LBL_1R_Bidding\PRICE_STEPS2.xlsx",engine='openpyxl')
PRICE_STEPS=PRICE_STEP.to_numpy()
#print(PRICE_STEPS)
## include for more scneario
for ii in range(1):
    pDA = pd.read_excel(r"C:\hybrid_bidding\software_module-20231023T230959Z-001\software_module\LBL_1R_Bidding\pricedata2.xlsx",sheet_name=ii,engine='openpyxl')
    genDA = pd.read_excel(r"C:\hybrid_bidding\software_module-20231023T230959Z-001\software_module\LBL_1R_Bidding\gendata2.xlsx",sheet_name=ii,engine='openpyxl')
    probSc = pd.read_excel(r"C:\hybrid_bidding\software_module-20231023T230959Z-001\software_module\LBL_1R_Bidding\probS2.xlsx",sheet_name=ii,engine='openpyxl')
    priceDA = pDA.to_numpy()
    #print(priceDA)
    gen_data= genDA.to_numpy()
    probS=probSc.to_numpy()
    #print(probS)
    from opt_prob import get_stoch_cvar_stairstep_prob_noRT
    bid,bid2=get_stoch_cvar_stairstep_prob_noRT(esr,gen,POI,eta_plus,eta_minus,dev_plus_addifzero,dev_minus_addifzero,priceDA,gen_data,probS,cvar_weight,cvar_confid,gridcharge,PRICE_STEPS,np,pyo,Time_p,nscen_breaks)
    #bids=pyo.value(bid.Pg_sc)
    obj_result = pyo.value(bid.profit)
    print("The o.f. is:",obj_result)

0
CPLEX 12.9.0.0# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 153800
  Number of variables: 115889
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CPLEX 12.9.0.0\x3a optimal integer solution; objective 816.7005835886645; 43 MIP simplex iterations; 0 branch-and-bound nodes
  Termination condition: optimal
  Id: 2
  Error rc: 0
  Time: 7.172562599182129
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [5]:
print(probS)
obj_result = pyo.value(bid.profit)
print("The o.f. is:",obj_result)

[[0.0840404 ]
 [0.00060606]
 [0.00181818]
 [0.00060606]
 [0.00060606]
 [0.00020202]
 [0.00040404]
 [0.00080808]
 [0.00020202]
 [0.00060606]
 [0.00080808]
 [0.00181818]
 [0.00020202]
 [0.00040404]
 [0.00060606]
 [0.0010101 ]
 [0.00121212]
 [0.00020202]
 [0.00181818]
 [0.0020202 ]
 [0.0840404 ]
 [0.00060606]
 [0.00181818]
 [0.00060606]
 [0.00060606]
 [0.00020202]
 [0.00040404]
 [0.00080808]
 [0.00020202]
 [0.00060606]
 [0.00080808]
 [0.00181818]
 [0.00020202]
 [0.00040404]
 [0.00060606]
 [0.0010101 ]
 [0.00121212]
 [0.00020202]
 [0.00181818]
 [0.0020202 ]
 [0.0840404 ]
 [0.00060606]
 [0.00181818]
 [0.00060606]
 [0.00060606]
 [0.00020202]
 [0.00040404]
 [0.00080808]
 [0.00020202]
 [0.00060606]
 [0.00080808]
 [0.00181818]
 [0.00020202]
 [0.00040404]
 [0.00060606]
 [0.0010101 ]
 [0.00121212]
 [0.00020202]
 [0.00181818]
 [0.0020202 ]
 [0.0840404 ]
 [0.00060606]
 [0.00181818]
 [0.00060606]
 [0.00060606]
 [0.00020202]
 [0.00040404]
 [0.00080808]
 [0.00020202]
 [0.00060606]
 [0.00080808]
 [0.00

In [9]:
    TPi = np.linspace(0, 48-1, num=48, dtype=int)
    TP=pyo.Set(initialize=TPi)
    #m.TP.pprint()
    print(pyo.value(bid.theta))
    TSi=np.linspace(0, 199, num=200, dtype=int)
    TS = pyo.Set(initialize=TSi)
    for s in range(200):
        for t in range(48):
            print(pyo.value(bid.e_p[s,t]))

0.1391145874548141
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
13.459791666666675
4.416979166666636
0.0
0.0
0.3460416666666717
0.8871874999999996
5.862187500000001
0.0
0.0
0.0
0.0
0.0
0.0
5.0978125
5.9823958333333955
0.0
0.0
9.232083333333332
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.22376514589168295
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.615624999999998
5.371979166666662
1.376114273216949
19.926250000000003
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.22376514589168295
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.615624999999998
5.371979166666662
13.640208333333327
7.662155939883625
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.615624999999998
5.371979166666662
1.376114273216949
19.926250000000003
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [22]:
print(b.shape)

print(b.T*np.ones((1,6)))

(1, 6)
[[0.34825095 0.34825095 0.34825095 0.34825095 0.34825095 0.34825095]
 [0.35310934 0.35310934 0.35310934 0.35310934 0.35310934 0.35310934]
 [0.35986368 0.35986368 0.35986368 0.35986368 0.35986368 0.35986368]
 [0.36509731 0.36509731 0.36509731 0.36509731 0.36509731 0.36509731]
 [0.36738826 0.36738826 0.36738826 0.36738826 0.36738826 0.36738826]
 [0.36738826 0.36738826 0.36738826 0.36738826 0.36738826 0.36738826]]


In [4]:
(nS,nT)=priceDA.shape
(nstep,nc)=PRICE_STEPS.shape
#print(PRICE_STEPS)
temp=PRICE_STEPS[:,6]
steprep=np.dot(temp,np.ones((nstep,nS)))

In [9]:
print(temp)


[-0.66148868 -0.65895768 -0.65626626 -0.64747904 -0.64129413]


In [7]:
print(nstep)

5


In [2]:
import numpy as np

array = [[1, 13, 6], [9, 4, 7], [19, 16, 2]]

# defining array
arr = np.array(array)

print('printing 0th row')
print(arr[0, :]*arr[:, 0])
c=arr[0, :]
print(c.shape)
print('printing 2nd column')
print(arr[:, 2])

# multiple columns or rows can be selected as well
print('selecting 0th and 1st row simultaneously')
print(arr[:,[0,1]])


printing 0th row
[  1 117 114]
(3,)
printing 2nd column
[6 7 2]
selecting 0th and 1st row simultaneously
[[ 1 13]
 [ 9  4]
 [19 16]]
